In [1]:
!pip install torch

In [31]:
#####################################################################################################
##### Linear regression using PyTorch built-ins ####
#####################################################################################################

# torch.nn package from PyTorch, which contains utility classes for building neural networks
import torch
import numpy as np

# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
outputs = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
outputs = torch.from_numpy(outputs)

In [32]:
# In PyTorch, the Dataset and DataLoader are essential components for handling data in the training of machine learning models.

# Dataset
# A Dataset in PyTorch stores the samples and their corresponding labels. It provides an interface for accessing the 
# training data. PyTorch supports two types of datasets: map-style and iterable-style datasets. Map-style datasets 
# implement the __getitem__() and __len__() protocols, while iterable-style datasets represent an iterable over a dataset.
    
# DataLoader
# On the other hand, a DataLoader is an iterable that abstracts the complexity of data handling. It wraps around a 
# Dataset and enables easy access to the samples, typically in minibatches. The DataLoader supports both map-style 
# and iterable-style datasets with single- or multi-process loading, allowing for customizing loading order and 
# optional automatic batching.
# The DataLoader is particularly useful for handling large datasets that cannot fit into memory, as well as for 
# performing data augmentation and preprocessing. It takes care of shuffling, sampling, batching, and using 
# multiprocessing to load the data, among other functionalities.    

# In summary, the Dataset is used to store and provide access to the training data, while the DataLoader is used to 
# iterate over the dataset in batches, handling various data loading complexities.


In [33]:
# 1. Load data

from torch.utils.data import TensorDataset

train_ds = TensorDataset(inputs, outputs)

# access a small section of the training data using the array indexing notation
train_ds[0:1]


(tensor([[73., 67., 43.]]), tensor([[56., 70.]]))

In [41]:
from torch.utils.data import DataLoader

# In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, 
# it shuffles the training data before creating batches. 
# Shuffling helps randomize the input to the optimization algorithm, 
# which can lead to faster reduction in the loss.

train_dl = DataLoader(train_ds, batch_size = 5, shuffle = True)


In [42]:
# 2. Define Model

model = torch.nn.Linear(3, 2)
print(model.weight)
print(model.bias)

# PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices 
# present in the model.
list(model.parameters())

Parameter containing:
tensor([[-0.3902,  0.1772, -0.5584],
        [ 0.4900, -0.2220, -0.4178]], requires_grad=True)
Parameter containing:
tensor([-0.2324, -0.4879], requires_grad=True)


[Parameter containing:
 tensor([[-0.3902,  0.1772, -0.5584],
         [ 0.4900, -0.2220, -0.4178]], requires_grad=True),
 Parameter containing:
 tensor([-0.2324, -0.4879], requires_grad=True)]

In [49]:
# Generate predictions
preds = model(inputs)
preds

# Import nn.functional
import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

loss = loss_fn(model(inputs), outputs)
print(loss)

tensor(56.5607, grad_fn=<MseLossBackward0>)


In [44]:
# Optimizer
# Instead of manually manipulating the model’s weights & biases using gradients, we can use the optimizer optim.SGD. 
# SGD stands for stochastic gradient descent. It is called stochastic because samples are selected in batches 
# (often with random shuffling) instead of as a single group.

# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [46]:
 # Repeat for given number of epochs
for epoch in range(100):

    # Train with batches of data
    for xb,yb in train_dl:

        # 1. Generate predictions
        pred = model(xb)

        # 2. Calculate loss
        loss = loss_fn(pred, yb)

        # 3. Compute gradients
        loss.backward()

        # 4. Update parameters using gradients
        opt.step()

        # 5. Reset the gradients to zero
        opt.zero_grad()

    # Print the progress
    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 100, loss.item()))
    
    # 1. We use the data loader defined earlier to get batches of data for every iteration.
    # 2. Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update, and 
    #    opt.zero_grad to reset the gradients to zero.
    # 3. We’ve also added a log statement which prints the loss from the last batch of data for every 10th epoch, 
    #    to track the progress of training. loss.item returns the actual value stored in the loss tensor.


Epoch [10/100], Loss: 140.8094
Epoch [20/100], Loss: 287.4706
Epoch [30/100], Loss: 101.9382
Epoch [40/100], Loss: 322.0182
Epoch [50/100], Loss: 130.6283
Epoch [60/100], Loss: 187.5990
Epoch [70/100], Loss: 116.1184
Epoch [80/100], Loss: 53.8673
Epoch [90/100], Loss: 14.5780
Epoch [100/100], Loss: 51.7240


In [47]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.1314,  72.1014],
        [ 78.0898,  96.5951],
        [126.1944, 139.1498],
        [ 26.6826,  47.5283],
        [ 91.4669, 105.8314],
        [ 58.1314,  72.1014],
        [ 78.0898,  96.5951],
        [126.1944, 139.1498],
        [ 26.6826,  47.5283],
        [ 91.4669, 105.8314],
        [ 58.1314,  72.1014],
        [ 78.0898,  96.5951],
        [126.1944, 139.1498],
        [ 26.6826,  47.5283],
        [ 91.4669, 105.8314]], grad_fn=<AddmmBackward0>)

In [48]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])